In [ ]:
import xgboost
import pickle
import numpy as np
import pandas as pd
import plotly.express as px
from src.

In [ ]:
transformed_data = pd.read_csv('../Data/data_pipeline_output_multi_entry_pnl_2022.csv')

In [ ]:
with open('../Models/entry_model_multi_entry.pkl', 'rb') as file:
    entry_model = pickle.load(file)

In [ ]:
pred_entry = entry_model.predict(transformed_data[entry_model.feature_names.values])

In [ ]:
# This table is avaiable to be loaded from DB
transformed_data['entry_appears'] = pred_entry

# User specifies the start and end date

e.g. user set timemachine to be on 2022-01-03, and want to see how well it does on 2022-12-31

In [ ]:
"""
User Input
"""
sim_start = '2022-01-03'
sim_end = '2022-12-31'
fund = 10000

In [ ]:
result, all_execution_history = run_simulation(
    starting_fund=10000,
    sim_start='2022-01-01',
    sim_end = '2022-12-31',
    transformed_data = transformed_data,
    refresh_cadence=60
)

In [ ]:
px.line(result, x="Date", y=['total_asset'])

### 1. Determine the dates where we are refreshing the pair trading profolio

In [ ]:
refresh_cadence = 60
unique_dates = transformed_data.Date[
     (transformed_data.Date >= sim_start) &
    (transformed_data.Date <= sim_end)
].unique()

total_date_num = len(unique_dates)

In [ ]:
idx_refresh_date = [refresh_cadence*i if (i*refresh_cadence)<=total_date_num else -1 for i in range(total_date_num)]
idx_refresh_date = np.array(idx_refresh_date)[np.array(idx_refresh_date)>=0]
refresh_dates = unique_dates[idx_refresh_date]
refresh_dates

On the dates above, we will get the pairs and calculate the daily pnls of the next 60 day

In [ ]:
# Get the data as of refresh date
refresh_date_data = transformed_data[
    transformed_data.Date.isin(refresh_dates)
]

# qualified_pairs = future_data.loc[future_data[future_data.entry_appears==1].groupby(["Date"])["stock2vec_cos_sim"].idxmax()]

In [ ]:
# Get the qualifying pairs for each date and the abs mean spread/std
refresh_date_data_pairs = refresh_date_data[
    (refresh_date_data.entry_appears == 1) & 
    (refresh_date_data.stock2vec_cos_sim>0.95)
][
    ['Date', 'Ticker_P1', 'Ticker_P2', 'abs_spread_mean_MA','abs_spread_std_MA', 'beta_P1', 'beta_P2']
]

# refresh_date_data_pairs = refresh_date_data.loc[refresh_date_data[refresh_date_data.entry_appears==1].groupby(["Date"])["stock2vec_cos_sim"].idxmax()]

In [ ]:
refresh_date_data_pairs.head(5)

### 2. Calculate PNLs at each date

for each date and pair above, calculate the daily pnl like below:

date, pnl, total asset

In [ ]:
"""
temp experimental input - will be in loop
"""
temp_date = '2022-01-03'

In [ ]:
refresh_date_rec_pair_tb = refresh_date_data_pairs[refresh_date_data_pairs.Date==temp_date]
num_paris_at_refresh_date = refresh_date_rec_pair_tb.shape[0]
fund_per_pair = fund/num_paris_at_refresh_date
fund_per_pair

In [ ]:
# Loop starts here for each pair
# for each pair calculate the gains
future_daily_pnl_tb = pd.DataFrame()

for idx in refresh_date_rec_pair_tb.index:
    temp = refresh_date_rec_pair_tb.loc[idx]
    
    # Get future 60 trade date price data
    future_price=transformed_data[
        (transformed_data.Ticker_P1 == temp.Ticker_P1) & 
        (transformed_data.Ticker_P2 == temp.Ticker_P2) &
        (transformed_data.Date >= temp.Date)
    ]\
    .reset_index(drop=True)\
    .loc[:refresh_cadence, ['Date','Ticker_P1', 'Ticker_P2','Close_P1', 'Close_P2']]

    cumu_pnl_pct = [0]
    for i in range(1,future_price.shape[0]):
        cumu_pnl_pct.append(
            trade_class.execute(
                vec1 = future_price.Close_P1.values[:i],
                vec2 = future_price.Close_P2.values[:i],
                dates= future_price.Date.values,
                beta_p1=temp.beta_P1,
                beta_p2=temp.beta_P2,
                base_fund=fund_per_pair
            ).final_pl_pct
        )
    future_price['cumu_pnl_pct'] = cumu_pnl_pct
    future_price['total_asset'] = fund_per_pair*(1+future_price['cumu_pnl_pct'])
    future_daily_pnl_tb = pd.concat(
        [future_daily_pnl_tb, future_price]
    )

In [ ]:
future_daily_pnl_tb_agg = future_daily_pnl_tb[[
    'Date',
    'total_asset'
]].groupby('Date').sum().reset_index()

In [ ]:
future_daily_pnl_tb_agg

In [ ]:
import plotly.express as px
fig = px.line(future_daily_pnl_tb_agg, x="Date", y=['total_asset'])

In [ ]:
fig